In [2]:
!pip install SchemDraw

## Типовой расчёт
Имеется конденсатор с сегнетоэлектриком. Найти зависимости ε и tg δ от температуры. Аппроксимировать эти зависимости сплайнами. Рассмотреть схему, состоящую из источника синусоидального напряжения, резистора и конденсатора. Построить на графиках табличные данные по зависимостям ε и tgδ от частоты, температуры, на тех же графиках нанести аппроксимированные кривые. Построить зависимости тока и напряжения на конденсаторе в зависимости от частоты и температуры. Разработать приложение с пользовательским
интерфейсом для изменения температуры.


Рассматриваеммый сегнетоэлектрик: BaTiO3 (Титанат бария)
Его зависимости ε и tgδ от частоты:<br>
<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1007%2Fs42452-020-2017-8/MediaObjects/42452_2020_2017_Fig4_HTML.png?as=webp" width="500">

#### Аппроксимируем данные и получим функции ε и tgδ от частоты при разных температурах

In [3]:
from numpy.polynomial.polynomial import polyval
import numpy as np
import numpy.linalg as la
import pandas as pd


def pol_model(x, y, n):
    m = len(x)
    A = np.zeros((m, n+1))
    for i in range(m):
        for j in range(n+1):
            A[i, j] = x[i]**j
    r = la.lstsq(A, y, rcond=-1)
    return r[0]

df_tgd_30 = pd.read_csv('tgd_30.csv')
df_tgd_130 = pd.read_csv('tgd_130.csv')
df_tgd_250 = pd.read_csv('tgd_250.csv')

x_tgd = np.linspace(2.5, 6, 1000)
a1 = pol_model(df_tgd_30.x, df_tgd_30.y, 10)
a2 = pol_model(df_tgd_130.x, df_tgd_130.y, 10)
a3 = pol_model(df_tgd_250.x, df_tgd_250.y, 10)
tgd_30_a = polyval(x_tgd, a1)
tgd_130_a = polyval(x_tgd, a2)
tgd_250_a = polyval(x_tgd, a3)

df_e_30 = pd.read_csv('e_30.csv')
df_e_130 = pd.read_csv('e_130.csv')
df_e_250 = pd.read_csv('e_250.csv')

x_e = np.linspace(2.18, 6.5, 1000)
a4 = pol_model(df_e_30.x, df_e_30.y, 10)
a5 = pol_model(df_e_130.x, df_e_130.y, 10)
a6 = pol_model(df_e_250.x, df_e_250.y, 10)
e_30_a = polyval(x_e, a4)
e_130_a = polyval(x_e, a5)
e_250_a = polyval(x_e, a6)

In [5]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from ipywidgets import widgets
import SchemDraw as schem
import SchemDraw.elements as e


def draw_scheme():
    s = schem.Drawing()
    V1 = s.add(e.SOURCE_I, label='V')
    s.add(e.RBOX, d='right', label='R')
    s.add(e.CAP, d='down', label='C')
    s.add(e.LINE, to=V1.start)
    s.draw()
    
def u(Um, F, t=1):
    w = 2*np.pi*F
    return Um*np.sin(w*t)

def c_calc(a, s, d, F, r, Um):
    U = u(Um, F)
    e0 = 8.85*(10**-12)
    e = polyval(np.log10(F), a)
    C = (e*e0*(s*(10**-4))) / d
    Xc = 1 / (2*np.pi*F*C)
    Zc = np.array([-complex(0, i) for i in Xc])
    Z = np.array([complex(r, i) for i in Xc])
    Ic = U / Z
    Uc = Ic * Zc
    return Ic, Uc


def main(*args):
    t = w_t.value
    um = w_um.value
    F = np.linspace(w_f.value[0]*1000, w_f.value[1]*1000, 1000)
    with out2:
        clear_output(wait=True)
        draw_scheme()
        fig = plt.figure(figsize=(15, 15))
        ax1 = fig.add_subplot(411)
        ax2 = fig.add_subplot(412)
        ax3 = fig.add_subplot(413)
        ax4 = fig.add_subplot(414)
        if t == 30:
            ax1.plot(df_tgd_30.x, df_tgd_30.y, 'ko')
            ax1.plot(x_tgd, tgd_30_a)
            ax2.plot(df_e_30.x, df_e_30.y, 'ko')
            ax2.plot(x_e, e_30_a)
            Ic, Uc = c_calc(a4, w_s.value, w_d.value, F, w_r.value, w_um.value)
        elif t == 130:
            ax1.plot(df_tgd_130.x, df_tgd_130.y, 'ko')
            ax1.plot(x_tgd, tgd_130_a)
            ax2.plot(df_e_130.x, df_e_130.y, 'ko')
            ax2.plot(x_e, e_130_a)
            Ic, Uc = c_calc(a5, w_s.value, w_d.value, F, w_r.value, w_um.value)
        elif t == 250:
            ax1.plot(df_tgd_250.x, df_tgd_250.y, 'ko')
            ax1.plot(x_tgd, tgd_250_a)
            ax2.plot(df_e_250.x, df_e_250.y, 'ko')
            ax2.plot(x_e, e_250_a)
            Ic, Uc = c_calc(a6, w_s.value, w_d.value, F, w_r.value, w_um.value)
        ax3.plot(F, Ic.real)
        ax4.plot(F, Uc.real)       
        ax1.set_title(f'Зависимость tgD от частоты при T = {t}')
        ax2.set_title(f'Зависимость epsilon от частоты при T ={t}')
        ax3.set_title(f'Зависимость тока на конденсаторе от частоты при T = {t}')
        ax4.set_title(f'Зависимость анпряжении на конденсаторе от частоты при T = {t}')
        ax1.set_xlabel('Freq'), ax2.set_xlabel('Freq'), ax3.set_xlabel('Freq'), ax4.set_xlabel('Freq')
        ax1.set_ylabel('tgD'), ax2.set_ylabel('Epsilon'), ax3.set_ylabel('Ic'), ax4.set_ylabel('Uc')
        plt.tight_layout()
        plt.show()


out1 = widgets.Output(layout={'width': '50%'})
out2 = widgets.Output(layout={'width': '50%'})
c_label = widgets.Label('Параметры конденсатора')
w_s = widgets.FloatSlider(description='s [см^2]', min=1, max=100, step=0.1, value=1)
w_d = widgets.FloatSlider(description='d [мм]', min=1, max=100, step=0.1, value=1)
u_label = widgets.Label('Амплитуда напряжения')
w_um = widgets.FloatSlider(description='Um [В]', min=1, max=300, step=0.1, value=220)
r_label = widgets.Label('Сопротивление резистора')
w_r = widgets.FloatSlider(description='R [Ом]', min=1, max=300, step=0.1, value=100)
f_label = widgets.Label('Диапозон частоты')
w_f = widgets.FloatRangeSlider(value=[1, 10], min=0, max=100, step=1, description='f [кГц]')
t_label = widgets.Label('Выбор температуры')
w_t = widgets.RadioButtons(description='$T [C^{\circ}]$', options={'30': 30,
                                         '130': 130,
                                         '250': 250})
button = widgets.Button(description='Запустить', button_style='primary')
button.on_click(main)
display(widgets.HBox([out1, out2]))
with out1:
    display(c_label, w_s, w_d, u_label, w_um, r_label, w_r, f_label, w_f, t_label, w_t, button)